<a href="https://colab.research.google.com/github/abhilasha-kumar/Connector/blob/master/search-models/search_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages and mounting drive

In [ ]:
!git clone https://github.com/abhilasha-kumar/Connector.git

Cloning into 'search-models'...
remote: Not Found
fatal: repository 'https://github.com/abhilasha-kumar/Connector/search-models.git/' not found


In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import heapq
import pandas as pd
import scipy.spatial.distance
import numpy as np
from numpy.random import randint
from sklearn.preprocessing import MinMaxScaler, normalize
from numpy.linalg import matrix_power
from scipy import stats
from sklearn import preprocessing
from numpy.random import choice
import json
import heapq
import json
import itertools
import sys
import scipy.spatial.distance
from scipy.special import softmax

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Importing embeddings

In [ ]:
# import glove embeddings
representations = {}
parentfolder = "/content/drive/My Drive/Connector-CogSci2021/Descriptive analyses and model predictions/"
representations['glove'] = pd.read_csv(parentfolder +"glove_embeddings.csv").transpose().values
vocab = pd.read_csv(parentfolder +"vocab_final.csv")
print(f"embeddings are shaped:", representations['glove'].shape)
print(f"vocab is {len(vocab)} words")

embeddings are shaped: (12218, 300)
vocab is 12218 words


# Defining search & RSA functions

In [ ]:
# import search functions
%run -i "/content/drive/My Drive/search-models/create_similarity_matrix.py"
%run -i "/content/drive/My Drive/search-models/create_graph.py"
%run -i "/content/drive/My Drive/search-models/random_walk.py"
%run -i "/content/drive/My Drive/search-models/union_intersection.py"
%run -i "/content/drive/My Drive/search-models/predication_vector.py"

# import RSA functions
%run -i "/content/drive/My Drive/search-models/compute_board_combos.py"
%run -i "/content/drive/My Drive/search-models/get_wordpair_list.py"
%run -i "/content/drive/My Drive/search-models/create_board_matrix.py"
%run -i "/content/drive/My Drive/search-models/literal_guesser.py"
%run -i "/content/drive/My Drive/search-models/pragmatic_speaker.py"

In [ ]:
parentfolder = "/content/drive/My Drive/Connector-CogSci2021/Descriptive analyses and model predictions/"
with open(parentfolder + 'boards.json', 'r') as json_file:
    boards = json.load(json_file)
board_combos = {board_name : compute_board_combos(board_name,boards) for board_name in boards.keys()}

# Constructing similarity matrix & Graph

In [ ]:
threshold = 0.4
sim_matrix_glove = create_similarity_matrix(representations['glove'])
Graph = create_graph(sim_matrix_glove, threshold)

# Running an example

In [ ]:
target = 'exam-algebra'
w1, w2 = target.split(sep = "-")
n_steps = 5
n_walks = 50

# computing union and intersection of independent walks
u, i = union_intersection(w1,w2, n_steps, n_walks, vocab)

union_candidates = list(u.vocab_word)
int_candidates = list(i.vocab_word)

print(f"{len(u)} items in union: {union_candidates}")
print(f"{len(i)} items in intersection: {int_candidates}")

clue = 'mathematics'
n = 5
wordpairlist = get_wordpair_list(board_combos, 'e1_board1_words')
target_index = wordpairlist.index(target)
clue_index = list(vocab["vocab_word"]).index(clue)

print(f"for wordpair {target} and clue {clue}")


## compute predictions on FULL vocab

print(f"ON FULL VOCAB")

a = literal_guesser('e1_board1_words', 'glove', list(vocab.vocab_word) , vocab, boards)[:,clue_index]
y = pragmatic_speaker('e1_board1_words', 18.858, 0.004, 'glove',list(vocab.vocab_word) , vocab, boards)

print("literal guesser prediction is:", wordpairlist[np.argmax(a)])
top = y[target_index,:].argsort()[-n:][::-1].tolist()
top_words = [list(vocab["vocab_word"])[x] for x in top]
print(f"top {n} prag speaker predictions are:", top_words)

print(f"ON CANDIDATES")

b_union = literal_guesser('e1_board1_words', 'glove', union_candidates, vocab, boards)[:, union_candidates.index(clue)]
c_union = pragmatic_speaker('e1_board1_words', 18.858, 0.004, 'glove', union_candidates, vocab, boards)

b_int = literal_guesser('e1_board1_words', 'glove', int_candidates, vocab, boards)[:, int_candidates.index(clue)]
c_int = pragmatic_speaker('e1_board1_words', 18.858, 0.004, 'glove', int_candidates, vocab, boards)

print("GUESSER candidate UNION prediction is:", wordpairlist[np.argmax(b_union)])
print("GUESSER candidate INTERSECTION prediction is:", wordpairlist[np.argmax(b_int)])

top = c_union[target_index,:].argsort()[-n:][::-1].tolist()
top_words = [list(u.vocab_word)[x] for x in top]
print(f"top {n} SPEAKER candidate UNION are:", top_words)

top = c_int[target_index,:].argsort()[-n:][::-1].tolist()
top_words = [list(i.vocab_word)[x] for x in top]
print(f"top {n} prag SPEAKER candidate INTERSECTION are:", top_words)

print(f"SIMPLE UNION/INTERSECTION")
print(f"considering top {n} nodes visited by union and intersection...")
print(f"highly visited nodes in the union: {union_candidates[:n]}")
print(f"highly visited nodes in the intersection: {int_candidates[:n]}")


314 items in union: ['mathematics', 'math', 'level', 'intend', 'chance', 'better', 'imply', 'apply', 'school', 'basic', 'learning', 'enable', 'choose', 'praise', 'processor', 'proceed', 'polygon', 'problem', 'probability', 'prediction', 'prior', 'precisely', 'prestigious', 'preschool', 'produce', 'preparation', 'primary', 'pronunciation', 'product', 'range', 'register', 'regard', 'refute', 'refuse', 'recognize', 'ratio', 'rather', 'quotient', 'professor', 'quantum', 'psychology', 'psychologist', 'psychiatric', 'proton', 'proposal', 'physics', 'plead', 'pack', 'perfect', 'particular', 'mostly', 'monastery', 'momentum', 'might', 'medicine', 'maximum', 'matrix', 'mathematical', 'market', 'manufacturer', 'mandatory', 'management', 'loop', 'logical', 'logic', 'multiplication', 'multiply', 'nature', 'occupy', 'particle', 'parameter', 'outdoor', 'orphanage', 'operate', 'offer', 'obligatory', 'need', 'objective', 'nursery', 'notion', 'none', 'never', 'neurology', 'regret', 'abstract', 'regulat

# Running through full dataset


In [ ]:
expdata = pd.read_csv(parentfolder + "final_board_clues_all.csv", encoding= 'unicode_escape')


allcombinations_df = pd.DataFrame(columns=['Board', 'Word1','Word2', 'wordpair'])


for board in board_combos:
    newdf = board_combos[board]
    newdf.insert(loc=0, column='Board', value=board)
    allcombinations_df = pd.concat([allcombinations_df, newdf])
